# Send and receive events from pipelines

In this notebook we show how to send and receive events from a pipeline from the SDK.

We start by creating a client with our personal access token.

In [30]:
from glassflow import GlassFlowClient

# Please edit this variable with your own personal access token from https://app.glassflow.dev/profile
personal_access_token = "< your-personal-access-token >"

In [31]:
client = GlassFlowClient(
    personal_access_token=personal_access_token
)

## Create Pipelines

Let's create the a space and a pipeline for the example

In [32]:
%%bash
echo "Writing transformation function ..."
cat <<EOT > transformation.py 
def handler(data, log):
    log.info(data)
    return data
EOT
echo "Done!"

Writing transformation function ...
Done!


In [33]:
space = client.create_space(name="tutorials")
pipeline = client.create_pipeline(
    name="example pipeline",
    space_id=space.id,
    transformation_file="transformation.py",
)

To send and receive data we need the pipeline's id and access token

In [34]:
PIPELINE_ID = pipeline.id
PIPELINE_ACCESS_TOKEN = pipeline.access_tokens[0]["token"]

print("Pipeline ID:", PIPELINE_ID)
print("Pipeline Access Token:", PIPELINE_ACCESS_TOKEN)

Pipeline ID: 09b17634-cbb3-43f4-8321-bb35b8736c0b
Pipeline Access Token: jWHT3XMQEp5nkKJRtTn2M2F3NzvzAay39pp3YyJDkxj4NtT7gNkBk8nHC5ZstFcFn77TJ8fA7PGVywf5fzJSSrAmwcyKDVmwNKTA9TgKndYvFp8xygrxSqJdkymbqGfy


## Send events

Now we can create source client to send events to the pipeline. 

In [35]:
from glassflow import PipelineDataSource

source = PipelineDataSource(
    pipeline_id=PIPELINE_ID,
    pipeline_access_token=PIPELINE_ACCESS_TOKEN
)

In [36]:
events = [
    {"name": "my-event", "idx": 1},
    {"name": "my-event", "idx": 2},
    {"name": "my-event", "idx": 3},
    {"name": "my-event", "idx": 4},
    {"name": "my-event", "idx": 5},
]

In [37]:
for event in events:
    source.publish(event)

## Receive events

Similarly, we can receive the events we sent by consuming from a sink client.

In [38]:
from glassflow import PipelineDataSink

sink = PipelineDataSink(
    pipeline_id=PIPELINE_ID,
    pipeline_access_token=PIPELINE_ACCESS_TOKEN
)

In [39]:
received_events = []
while True:
    res = sink.consume()

    if res.status_code == 200:
        event = res.json()
        print(event)
        received_events.append(event)

    if len(received_events) == 5:
        break

{'idx': 1, 'name': 'my-event'}
{'idx': 2, 'name': 'my-event'}
{'idx': 3, 'name': 'my-event'}
{'idx': 4, 'name': 'my-event'}
{'idx': 5, 'name': 'my-event'}


## Clean-up

Delete the pipelines and spaces we created:

In [40]:
pipeline.delete()
space.delete()